In [12]:
!pip install num2words

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [13]:
!pip install av

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [28]:
!pip install opencv-python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 18.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.1
    Uninstalling numpy-2.3.1:
      Successfully uninstalled numpy-2.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import time

In [8]:
import psutil
import os
def get_memory_usage_mb():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)  # in MB

In [9]:
SAFETY_PROMPTS = [
    "Analyze this industrial/construction site video. Identify any safety violations including: people without hardhats, people without safety vests, unauthorized personnel in restricted zones, or vehicles too close to workers. List all safety hazards you detect.",
    "You are a safety inspector. Examine this video for workplace safety violations. Report: 1) Missing PPE (hardhats, safety vests, safety boots), 2) Restricted area violations, 3) Unsafe vehicle-pedestrian distances. Be specific about what you observe.",
    "Industrial safety check: Look for safety hazards in this video including workers without proper protective equipment, people in danger zones, and unsafe proximity between vehicles and personnel. Provide a safety assessment.",
]

SAFETY_QUESTIONS = [
    "Are there any people in the video not wearing hardhats?",
    "Do all visible workers have safety vests on?",
    "Is anyone in the video located inside a restricted or unauthorized zone?",
    "Are there any vehicles dangerously close to workers in the video?",
    "Is anyone missing required PPE like safety boots, vests, or helmets?",
    "Is there an unsafe distance between any vehicle and a person?",
    "Are all workers following proper industrial safety protocols?",
    "Is anyone working in an area they should not be in?",
    "Are there any visible safety violations in this video?",
    "Does the video show any hazards that could pose a threat to worker safety?"
]

## Post-video Inference

### SmolVLM2-500M-Video-Instruct

In [17]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText

In [18]:
model_id = "HuggingFaceTB/SmolVLM2-500M-Video-Instruct"

processor = AutoProcessor.from_pretrained(model_id)

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.float32
).to("cpu")

In [19]:
def run_safety_analysis(video_path, prompts):
    results = {}
    for prompt in prompts:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "video", "path": video_path},
                    {"type": "text", "text": prompt},
                ],
            }
        ]
        inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cpu")

        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=400)
            response = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
            
            if "Assistant:" in response:
                clean_response = response.split("Assistant:")[-1].strip()
            else:
                clean_response = response
            
            results[prompt] = clean_response
    
    return results

In [20]:
video_file = "images/camera 1.mp4"
start_time = time.time()
before_mem = get_memory_usage_mb()

findings = run_safety_analysis(video_file, SAFETY_QUESTIONS)

for q, ans in findings.items():
    print(f"\nPrompt: {q}\nResponse: {ans}\n{'-'*50}")

end_time = time.time()
after_mem = get_memory_usage_mb()
print(f"Memory used (CPU RAM): {after_mem - before_mem:.2f} MB")

print(f"Total Time: {end_time - start_time:.2f} seconds")


Prompt: Are there any people in the video not wearing hardhats?
Response: Yes, there are people in the video who are not wearing hardhats, such as a person in a black shirt and another in a blue shirt.
--------------------------------------------------

Prompt: Do all visible workers have safety vests on?
Response: Yes, all visible workers are wearing safety vests.
--------------------------------------------------

Prompt: Is anyone in the video located inside a restricted or unauthorized zone?
Response: Yes, there is a person in a yellow hard hat and a white shirt walking on the sidewalk, which is not allowed in the restricted zone.
--------------------------------------------------

Prompt: Are there any vehicles dangerously close to workers in the video?
Response: Yes, there are several vehicles, including a yellow bus and a yellow truck, that are very close to the workers.
--------------------------------------------------

Prompt: Is anyone missing required PPE like safety boots

## Real-time Video Inference

### SmolVLM-500M-Instruct

In [22]:
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq
import cv2

In [2]:
processor2 = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-500M-Instruct")
model2 = AutoModelForVision2Seq.from_pretrained("HuggingFaceTB/SmolVLM-500M-Instruct")

In [23]:
from PIL import Image
def analyze_frame(frame, prompt):
    """Run inference on a single frame with given prompt."""
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": frame},
                {"type": "text", "text": prompt},
            ],
        }
    ]
    inputs = processor2.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to("cpu")
    
    with torch.no_grad():
        output_ids = model2.generate(**inputs, max_new_tokens=100)
        response = processor2.batch_decode(output_ids, skip_special_tokens=True)[0]
        
        # Clean the response
        if "Assistant:" in response:
            clean_response = response.split("Assistant:")[-1].strip()
        else:
            clean_response = response
            
    return clean_response

In [26]:
def run_realtime_analysis(video_path, prompts, frame_interval=30):
    """
    Processes video frame by frame and runs safety analysis prompts.
    Args:
        video_path (str): Path to the video file.
        prompts (list): List of textual safety questions/prompts.
        frame_interval (int): Process every Nth frame (default=30 ~ 1 sec at 30fps).
    """    
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0
    results = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        if frame_count % frame_interval == 0:
            frame_start_time = time.time()
            
            # Timestamp in seconds
            timestamp = frame_count / fps
            # Convert frame to PIL image
            pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            
            for prompt in prompts:
                prompt_start_time = time.time()
                
                messages = [
                    {
                        "role": "user",
                        "content": [
                            {"type": "image", "image": pil_frame},
                            {"type": "text", "text": prompt},
                        ],
                    }
                ]
                inputs = processor2.apply_chat_template(
                    messages,
                    add_generation_prompt=True,
                    tokenize=True,
                    return_dict=True,
                    return_tensors="pt"
                ).to("cpu")
                
                with torch.no_grad():
                    output_ids = model2.generate(**inputs, max_new_tokens=200)
                    response = processor2.batch_decode(output_ids, skip_special_tokens=True)[0]
                    
                    # Clean the response
                    if "Assistant:" in response:
                        clean_response = response.split("Assistant:")[-1].strip()
                    else:
                        clean_response = response
                
                prompt_time = time.time() - prompt_start_time
                
                # Print findings in real-time
                print(f"\n[Time: {timestamp:.2f}s] [Processing: {prompt_time:.2f}s] Prompt: {prompt}\nResponse: {clean_response}\n{'-'*60}")
                
                results.append({
                    "time": timestamp,
                    "prompt": prompt,
                    "response": clean_response,
                    "processing_time": prompt_time
                })
            
            frame_total_time = time.time() - frame_start_time
            print(f"\n>>> FRAME TOTAL TIME: {frame_total_time:.2f}s <<<\n{'='*60}")
        
        frame_count += 1
    
    cap.release()
    return results

In [27]:
video_file = "images/camera 1.mp4"
start_time = time.time()
before_mem = get_memory_usage_mb()
findings = run_realtime_analysis(video_file, SAFETY_QUESTIONS)

end_time = time.time()
after_mem = get_memory_usage_mb()
print(f"Memory used (CPU RAM): {after_mem - before_mem:.2f} MB")

print(f"Total Time: {end_time - start_time:.2f} seconds")


[Time: 0.00s] [Processing: 7.99s] Prompt: Are there any people in the video not wearing hardhats?
Response: Yes, there are people in the video who are not wearing hardhats.
------------------------------------------------------------

[Time: 0.00s] [Processing: 7.44s] Prompt: Do all visible workers have safety vests on?
Response: Yes, all visible workers are wearing safety vests.
------------------------------------------------------------

[Time: 0.00s] [Processing: 7.98s] Prompt: Is anyone in the video located inside a restricted or unauthorized zone?
Response: The provided facts do not mention any restricted or unauthorized zones, so it cannot be determined from the image.
------------------------------------------------------------

[Time: 0.00s] [Processing: 7.56s] Prompt: Are there any vehicles dangerously close to workers in the video?
Response: Yes, there are vehicles dangerously close to workers in the video.
------------------------------------------------------------

[Time

## Visual Analysis of tested models

In [29]:
frame_durations = []

for finding in findings:
    frame_durations.append(finding['processing_time'])

In [36]:
!pip install plotly
!pip install pandas
!pip install nbformat

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 21.9 MB/s eta 0:00:00 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached jsonschema-4.25.0-py3-none-any.whl.metadata (7.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached jsonschema-4.25.0-py3-none-any.whl (89 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl (18 kB)
Using cached referencing-0.36.2-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [nbformat]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import plotly.graph_objects as go

# Example results (mock timings for each model, replace with real timings if available)
model_times = {
    "SmolVLM-500M-Instruct": 1352.79,   # seconds
    "SmolVLM-500M-Video-Instruct": 134.95,   # seconds
}

frame_durations = [7.988126039505005, 7.441399097442627, 7.982293128967285, 7.5592567920684814, 8.03861689567566, 7.494975805282593, 7.29808497428894, 7.6557090282440186, 7.522607803344727, 7.96260404586792, 7.750208139419556, 7.4618470668792725, 8.080986976623535, 7.594213962554932, 8.078094005584717, 7.471828937530518, 7.432935953140259, 7.727540969848633, 7.57755708694458, 7.9281980991363525, 7.848785877227783, 7.561712026596069, 7.861083984375, 7.591317892074585, 7.698180198669434, 7.303893089294434, 7.331212997436523, 7.737868070602417, 7.665045976638794, 7.969285011291504, 8.023762941360474, 7.790822267532349, 8.063060998916626, 7.863956928253174, 8.8474440574646, 8.509550094604492, 7.703817844390869, 9.44123888015747, 7.7800681591033936, 8.179249048233032, 7.926742076873779, 7.6156721115112305, 7.706362962722778, 8.026015996932983, 8.09115719795227, 7.432284832000732, 7.362820148468018, 7.664441823959351, 7.46610689163208, 8.337284803390503, 7.7466208934783936, 7.434203147888184, 7.780443906784058, 7.620444059371948, 8.114219903945923, 8.237276315689087, 7.859813213348389, 8.141057014465332, 7.842684984207153, 8.243405103683472, 8.117693901062012, 7.769243955612183, 8.058409929275513, 7.908437967300415, 8.206146001815796, 7.8853981494903564, 7.678642988204956, 8.002275228500366, 7.884188175201416, 8.27356481552124, 8.16845989227295, 7.927879095077515, 8.396631002426147, 8.271772146224976, 8.538717269897461, 7.94507098197937, 7.869871139526367, 8.220475912094116, 8.183341026306152, 8.485485792160034, 8.067385911941528, 7.576082944869995, 7.760834217071533, 7.502058982849121, 7.895010948181152, 7.847278118133545, 7.64295506477356, 7.468363046646118, 7.326026916503906, 7.554950952529907, 7.522262096405029, 7.2473180294036865, 8.312555074691772, 7.7969069480896, 7.958584785461426, 7.484839200973511, 7.212639093399048, 8.441498041152954, 7.4144861698150635, 7.807614088058472, 8.899427890777588, 7.474862813949585, 7.56626296043396, 7.460575103759766, 8.773640155792236, 8.168505191802979, 8.89005994796753, 8.52662706375122, 8.212888240814209, 9.028141021728516, 8.044249057769775, 8.015451908111572, 8.4667649269104, 8.193203210830688, 8.632941246032715, 8.194445133209229, 7.364217758178711, 8.40737271308899, 7.593324184417725, 7.59186577796936, 7.642949819564819, 7.395264148712158, 8.225261688232422, 8.124586343765259, 9.041513919830322, 7.96804404258728, 8.428974628448486, 8.172607183456421, 9.876458168029785, 8.352301120758057, 8.971984148025513, 7.772378206253052, 7.736438035964966, 9.47153902053833, 7.98571515083313, 7.667093992233276, 7.4053709506988525, 8.079293966293335, 8.409473896026611, 8.299506902694702, 8.913743019104004, 9.354561805725098, 8.356421947479248, 8.120311975479126, 8.766600131988525, 7.721855401992798, 7.385278940200806, 7.799396991729736, 7.699795246124268, 8.380215167999268, 7.843181133270264, 7.53987717628479, 7.8752968311309814, 7.516602993011475, 8.467005014419556, 7.662873029708862, 7.376661062240601, 7.792434930801392, 7.703174829483032, 8.011789798736572, 7.917418956756592, 7.532609939575195, 7.882156133651733, 7.797562837600708, 8.332176208496094, 7.742718935012817, 7.392119884490967, 7.731805086135864, 7.608674764633179, 8.277629375457764]
frames = list(range(1, len(frame_durations) + 1))

# --- Total Times Comparison ---
fig_total = go.Figure([
    go.Bar(x=list(model_times.keys()), y=list(model_times.values()), marker_color="skyblue")
])
fig_total.update_layout(title="Total Inference Time Comparison",
                        xaxis_title="Models",
                        yaxis_title="Total Time (s)")

fig_total.show()


In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

frame_durations = [7.988126039505005, 7.441399097442627, 7.982293128967285, 7.5592567920684814, 8.03861689567566, 7.494975805282593, 7.29808497428894, 7.6557090282440186, 7.522607803344727, 7.96260404586792, 7.750208139419556, 7.4618470668792725, 8.080986976623535, 7.594213962554932, 8.078094005584717, 7.471828937530518, 7.432935953140259, 7.727540969848633, 7.57755708694458, 7.9281980991363525, 7.848785877227783, 7.561712026596069, 7.861083984375, 7.591317892074585, 7.698180198669434, 7.303893089294434, 7.331212997436523, 7.737868070602417, 7.665045976638794, 7.969285011291504, 8.023762941360474, 7.790822267532349, 8.063060998916626, 7.863956928253174, 8.8474440574646, 8.509550094604492, 7.703817844390869, 9.44123888015747, 7.7800681591033936, 8.179249048233032, 7.926742076873779, 7.6156721115112305, 7.706362962722778, 8.026015996932983, 8.09115719795227, 7.432284832000732, 7.362820148468018, 7.664441823959351, 7.46610689163208, 8.337284803390503, 7.7466208934783936, 7.434203147888184, 7.780443906784058, 7.620444059371948, 8.114219903945923, 8.237276315689087, 7.859813213348389, 8.141057014465332, 7.842684984207153, 8.243405103683472, 8.117693901062012, 7.769243955612183, 8.058409929275513, 7.908437967300415, 8.206146001815796, 7.8853981494903564, 7.678642988204956, 8.002275228500366, 7.884188175201416, 8.27356481552124, 8.16845989227295, 7.927879095077515, 8.396631002426147, 8.271772146224976, 8.538717269897461, 7.94507098197937, 7.869871139526367, 8.220475912094116, 8.183341026306152, 8.485485792160034, 8.067385911941528, 7.576082944869995, 7.760834217071533, 7.502058982849121, 7.895010948181152, 7.847278118133545, 7.64295506477356, 7.468363046646118, 7.326026916503906, 7.554950952529907, 7.522262096405029, 7.2473180294036865, 8.312555074691772, 7.7969069480896, 7.958584785461426, 7.484839200973511, 7.212639093399048, 8.441498041152954, 7.4144861698150635, 7.807614088058472, 8.899427890777588, 7.474862813949585, 7.56626296043396, 7.460575103759766, 8.773640155792236, 8.168505191802979, 8.89005994796753, 8.52662706375122, 8.212888240814209, 9.028141021728516, 8.044249057769775, 8.015451908111572, 8.4667649269104, 8.193203210830688, 8.632941246032715, 8.194445133209229, 7.364217758178711, 8.40737271308899, 7.593324184417725, 7.59186577796936, 7.642949819564819, 7.395264148712158, 8.225261688232422, 8.124586343765259, 9.041513919830322, 7.96804404258728, 8.428974628448486, 8.172607183456421, 9.876458168029785, 8.352301120758057, 8.971984148025513, 7.772378206253052, 7.736438035964966, 9.47153902053833, 7.98571515083313, 7.667093992233276, 7.4053709506988525, 8.079293966293335, 8.409473896026611, 8.299506902694702, 8.913743019104004, 9.354561805725098, 8.356421947479248, 8.120311975479126, 8.766600131988525, 7.721855401992798, 7.385278940200806, 7.799396991729736, 7.699795246124268, 8.380215167999268, 7.843181133270264, 7.53987717628479, 7.8752968311309814, 7.516602993011475, 8.467005014419556, 7.662873029708862, 7.376661062240601, 7.792434930801392, 7.703174829483032, 8.011789798736572, 7.917418956756592, 7.532609939575195, 7.882156133651733, 7.797562837600708, 8.332176208496094, 7.742718935012817, 7.392119884490967, 7.731805086135864, 7.608674764633179, 8.277629375457764]
frames = list(range(1, len(frame_durations) + 1))


num_prompts = 10
num_frames = len(frame_durations) // num_prompts

# reshape into (frames, prompts)
durations_array = np.array(frame_durations).reshape(num_frames, num_prompts)

# average across prompts → one value per frame
frame_avg_times = durations_array.mean(axis=1)

frames = list(range(1, num_frames + 1))

# --- Subplots: Frame-wise Average + All datapoints ---
fig = make_subplots(rows=1, cols=2, subplot_titles=("Frame-wise Average Times", "All Prompts per Frame"))

# Left graph: frame-wise averages
fig.add_trace(
    go.Scatter(x=frames, y=frame_avg_times, mode="lines+markers", line=dict(color="orange"), name="Average per Frame"),
    row=1, col=1
)

# Right graph: all datapoints per frame
for i in range(num_frames):
    fig.add_trace(
        go.Scatter(x=[frames[i]]*num_prompts, y=durations_array[i], mode="markers", name=f"Frame {frames[i]}", showlegend=False),
        row=1, col=2
    )

fig.update_xaxes(title_text="Frame", row=1, col=1)
fig.update_yaxes(title_text="Avg Time per Frame (s)", row=1, col=1)

fig.update_xaxes(title_text="Frame", row=1, col=2)
fig.update_yaxes(title_text="Time per Prompt (s)", row=1, col=2)

fig.update_layout(title="SmolVLM Frame-wise Inference Times")

fig.show()
